<style>
    .highlight-text {
        font-family: 'Arial', sans-serif;
        font-size: 3em; /* Increased for heading prominence */
        font-weight: bold;
        background: linear-gradient(45deg, #3498db, #8e44ad);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        text-shadow: 3px 3px 6px rgba(0, 0, 0, 0.3); /* Enhanced text shadow */
        display: block;
        text-align: center; /* Center the text */
        padding: 15px 20px; /* Increased padding for heading feel */
        border-radius: 10px;
        background-color: #153160ff; /* Retained background color */
        box-shadow: 0 4px 12px 4px rgba(0, 0, 0); /* Added box shadow for depth */
        margin: 20px auto; /* Centered with auto margins */
        max-width: 80%; /* Prevents overly wide heading */
        transition: transform 0.3s ease, box-shadow 0.3s ease; /* Smooth hover effects */
    }

    .highlight-text:hover {
        transform: scale(1.05); /* Subtle zoom on hover */
        box-shadow: 0 6px 16px rgba(0, 0, 0, 0.3); /* Enhanced shadow on hover */
    }
</style>

<span class="highlight-text">Text Preprocessing</span>


<div style="max-width: 650px; margin: 40px auto; padding: 25px; border-radius: 10px; text-align: center; background: linear-gradient(135deg, #000000ff, #000000ff); border: 2px solid #ffffff; box-shadow: 0 6px 20px rgba(0, 0, 0, 0.15);" onmouseover="this.style.transform='translateY(-3px)'; this.style.boxShadow='0 8px 25px rgba(0, 0, 0, 0.2)'" onmouseout="this.style.transform='translateY(0)'; this.style.boxShadow='0 6px 20px rgba(0, 0, 0, 0.15)'">
  <h3 style="font-size: 24px; font-weight: 700; color: #212121; margin-bottom: 20px; font-family: 'Lato', Arial, sans-serif; letter-spacing: 1px;">
    <strong style = color:white>Author Name:</strong>
    <a href="https://www.linkedin.com/in/yousuf-shah-7ba9492b4/" target="_blank" style="color: #daff06ff; text-decoration: none; font-weight: bold;">Yousuf Shah</a>
  </h3>
  <table style="margin: 0 auto; border-spacing: 20px;">
    <tr>
      <td>
        <a href="https://www.linkedin.com/in/yousuf-shah-7ba9492b4/" target="_blank">
          <img src="https://img.shields.io/badge/LinkedIn-Profile-blue?style=for-the-badge&logo=linkedin" alt="LinkedIn" style="height: 55px; border-radius: 8px; background: linear-gradient(45deg, #000000, #000000); padding: 5px;"/>
        </a>
      </td>
      <td>
        <a href="https://yousfshah.github.io/Portfolio_Website/" target="_blank">
          <img src="https://img.shields.io/badge/Portfolio-Website-blue?style=for-the-badge&logo=link" alt="Portfolio Website" style="height: 55px; border-radius: 8px; background: linear-gradient(45deg, #000000, #000000); padding: 5px;"/>
        </a>
      </td>
    </tr>
    <tr>
      <td>
        <a href="https://github.com/Yousfshah" target="_blank">
          <img src="https://img.shields.io/badge/GitHub-Profile-green?style=for-the-badge&logo=github" alt="GitHub" style="height: 55px; border-radius: 8px; background: linear-gradient(45deg, #000000, #000000); padding: 5px;"/>
        </a>
      </td>
      <td>
        <a href="https://www.kaggle.com/yousufshah" target="_blank">
          <img src="https://img.shields.io/badge/Kaggle-Profile-orange?style=for-the-badge&logo=kaggle" alt="Kaggle" style="height: 55px; border-radius: 8px; background: linear-gradient(45deg, #000000, #000000); padding: 5px;"/>
        </a>
      </td>
    </tr>
    <tr>
        <td>
        <a href="https://yousfshah.github.io/Blogging/" target="_blank">
          <img src="https://img.shields.io/badge/Blog-Posts-purple?style=for-the-badge&logo=blogger" alt="Blog" style="height: 55px; border-radius: 8px; background: linear-gradient(45deg, #000000, #000000); padding: 5px;"/>
        </a>
      </td>
    </tr>
  </table>
</div>


<style>
  /* General Reset */
  * {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
  }

  /* Contact Info Section */
  .contact-info {
    max-width: 700px; /* Slightly wider for better spacing */
    margin: 30px auto;
    padding: 25px;
    border-radius: 20px;
    text-align: center;
    background: linear-gradient(135deg, #ffffffff, #000d0cff); /* Dark gradient for modern look */
    border: 3px solid #000000ff; /* Vibrant border */
    box-shadow: 0 8px 25px rgba(0, 0, 0, 0.5), inset 0 0 10px rgba(255, 255, 255, 0.2); /* 3D effect */
    transition: transform 0.3s ease, box-shadow 0.3s ease;
  }

  .contact-info:hover {
    transform: translateY(-5px); /* Lift effect on hover */
    box-shadow: 0 12px 30px rgba(119, 0, 0, 0.6), inset 0 0 15px rgba(122, 0, 86, 0.3);
  }

  /* Section Titles */
  .section-title {
    font-size: 24px;
    font-weight: bold;
    color: #0a0101ff; /* White for contrast */
    margin-bottom: 20px;
    text-shadow: 0 0 8px rgba(2, 62, 19, 0.7); /* Glowing effect */
    font-family: 'Roboto', Arial, sans-serif;
  }

  .section-title a {
    color: #02303cff; /* Vibrant cyan */
    text-decoration: none;
    transition: color 0.3s ease, text-shadow 0.3s ease;
  }

  .section-title a:hover {
    color: #340000ff; /* Coral on hover */
    text-shadow: 0 0 12px rgba(84, 0, 76, 0.8);
  }

  /* Table for Social Links */
  .contact-info table {
    margin: 0 auto;
    border-spacing: 20px; /* Increased for better separation */
  }

  /* Social Media Badges */
  .contact-info img {
    height: 60px; /* Slightly larger badges */
    border-radius: 12px;
    transition: all 0.3s ease;
    box-shadow: 0 4px 15px rgba(0, 0, 0, 0.3); /* Initial shadow */
    background: linear-gradient(45deg, #000000ff, #4d4d4dff); /* Card-like background */
    padding: 5px; /* Subtle padding for card effect */
  }

  .contact-info img:hover {
    transform: scale(1.2) rotate(5deg); /* Scale and slight rotation */
    box-shadow: 0 8px 20px rgba(0, 0, 0, 0.5), 0 0 15px rgba(176, 192, 0, 0.7); /* Glowing shadow */
    background: linear-gradient(45deg, #2b0353ff, #610012ff); /* Gradient on hover */
  }
</style>

<div class="contact-info">
  <h3 class="section-title">
    <strong>Author Name:</strong>
    <a href="https://www.linkedin.com/in/yousuf-shah-7ba9492b4/" target="_blank">Yousuf Shah</a>
  </h3>
  <table>
    <tr>
      <td>
        <a href="https://www.linkedin.com/in/yousuf-shah-7ba9492b4/" target="_blank">
          <img src="https://img.shields.io/badge/LinkedIn-Profile-blue?style=for-the-badge&logo=linkedin" alt="LinkedIn" />
        </a>
      </td>
      <td>
        <a href="https://yousfshah.github.io/Portfolio_Website/" target="_blank">
          <img src="https://img.shields.io/badge/Portfolio_Website-Website-blue?style=for-the-badge&logo=link" alt="Portfolio Website" />
        </a>
      </td>
    </tr>
    <tr>
      <td>
        <a href="https://github.com/Yousfshah" target="_blank">
          <img src="https://img.shields.io/badge/GitHub-Profile-green?style=for-the-badge&logo=github" alt="GitHub" />
        </a>
      </td>
      <td>
        <a href="https://www.kaggle.com/yousufshah" target="_blank">
          <img src="https://img.shields.io/badge/Kaggle-Profile-orange?style=for-the-badge&logo=kaggle" alt="Kaggle" />
        </a>
      </td>
    </tr>
  </table>
</div>


<div style="max-width: 800px; margin: 2.5rem auto; padding: 2rem 2.5rem; background: linear-gradient(135deg, #8d8d8dff, #9c9b9bff); box-shadow: 0 0.75rem 1.5rem rgba(0, 0, 0, 0.08); font-family: 'Segoe UI', 'Lato', sans-serif; color: #111; border-left: 6px solid #0055ff; border-radius: 0.5rem;">

  <h3 style="font-size: 1.8rem; font-weight: 700; margin-bottom: 1.2rem; color: #000000ff;">
    📌 Why You Should Practice This Notebook
  </h3>

  <p style="font-size: 1.05rem; line-height: 1.7; margin-bottom: 1rem; color:#000; font-weight: bold;">
    If you're serious about mastering NLP, this notebook gives you a hands-on, end-to-end experience with real-world text preprocessing — the foundation of every NLP project.
    You'll learn to clean, normalize, and prepare raw text using both industry-standard tools (<strong>spaCy</strong> + <strong>NLTK</strong>) and best practices followed in real ML pipelines.
  </p>

  <ul style="font-size: 1.05rem; line-height: 1.7; padding-left: 1.2rem; margin-bottom: 1rem; color:#000; font-weight: bold;">
    <li>✅ Practical code examples</li>
    <li>✅ Reusable functions</li>
    <li>✅ Cleaned input ready for ML, BERT, or vectorization</li>
  </ul>

  <p style="font-size: 1.05rem; line-height: 1.7; font-weight: 600; color: #ab1700ff;">
    Good models start with great preprocessing.<br>
  </p>

</div>


## 1️⃣ **Lowercasing**

### 🔍 **Why:**
- Words like "Good", "GOOD", and "good" are semantically identical but are treated as separate tokens if not lowercased.


### 📌 **Real-World:**
- In a product review system, "Excellent" and "excellent" should be considered the same sentiment word.

In [80]:
text = "Good Morning EVERYONE! Let's Start Our NLP Journey."
lowercased = text.lower()

print("🔵 Original Text: ", text, "\n")
print("🟢 Lower Case Text: ", lowercased)

🔵 Original Text:  Good Morning EVERYONE! Let's Start Our NLP Journey. 

🟢 Lower Case Text:  good morning everyone! let's start our nlp journey.


---

## 2️⃣ **Remove HTML Tags**

### 🔍 **Why:**
- HTML is markup used for layout, not meaning.

- If you're scraping websites (news, blogs), tags like div tag, span tag add noise.

### 📌 **Real-World:**
- When extracting articles from news sites, you'll find lots of formatting tags. Models get confused by p tag, a href tag etc.

In [97]:
from bs4 import BeautifulSoup
import regex as re

# sample HTML text
text = "<div>Hello <b>World</b>! NLP is <i>awesome</i>.</div>"

# Using BeautifulSoup to remove HTML tags
cleaned = BeautifulSoup(text, "html.parser").get_text()

# Using regex to remove HTML tags
re_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>" # # Remove all HTML tags from a string
cleaned_re = re.sub(re_pattern, '', text)
# cleaned = cleaned.strip()

print("🔵 Original Text: ", text, "\n")
print("🟢 Remove HTML Tags with BS4: ", cleaned, "\n")
print("🔴 Remove HTML Tags with regex: ", cleaned_re)

🔵 Original Text:  <div>Hello <b>World</b>! NLP is <i>awesome</i>.</div> 

🟢 Remove HTML Tags with BS4:  Hello World! NLP is awesome. 

🔴 Remove HTML Tags with regex:  Hello World! NLP is awesome.


---

## 3️⃣ **Remove URLs**

### 🔍 **Why:**
- URLs usually don’t carry semantic meaning.

- They are high variance strings (each one is unique) and hurt model generalization.

### 📌 **Real-World:**
- In a tweet like “Check this out 👉 https://xyz.com”, we care more about the sentiment or emotion, not the URL itself.

In [ ]:
# it will remove any type of URL from the text just apply on your text
import regex as re

# Example usage:
text = "Visit our website at https://www.example.com/page.html or check out www.another-site.org. You can also find info at mydomain.net/info."

# Regex pattern to match various URL formats (http, https, www, without scheme)
url_pattern = re.compile(
    r'https?://[^\s/$.?#].[^\s]*'  # Matches http/https URLs
    r'|www\.[^\s/$.?#].[^\s]*'    # Matches www. URLs
    r'|[a-zA-Z0-9.-]+\.[a-zA-Z]{2,6}(?:/[^\s]*)?' # Matches domain-only URLs like example.com
)

# Remove URLs from the text
text_without_urls= url_pattern.sub(r'', text)

print("🔵 Original Text: ", text, "\n")
print("🟢 Text Without URL: ", text_without_urls)

🔵 Original:  Visit our website at https://www.example.com/page.html or check out www.another-site.org. You can also find info at mydomain.net/info. 

🟢 Text Without URL:  Visit our website at  or check out  You can also find info at 


---

## 4️⃣ **Remove Punctuation**

### 🔍 **Why:**
-  Punctuation marks like `!`, `.`, `?` create unnecessary tokens.

- Removing them helps reduce noise, especially in tasks like text classification or topic modeling.

| Method                               | Removes ASCII | Removes Unicode | Customizable |
| ------------------------------------ | ------------- | --------------- | ------------ |
| `string.punctuation` + `translate()` | ✅ Yes         | ❌ No            | ❌ Limited    |
| `re.sub(r'\p{P}+', '', text)`       | ✅ Yes         | ✅ Yes           | ✅ Yes        |

- string.punctuation only covers ASCII punctuation & returns: <div style=color:red;>!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~</div>

- Unicode symbols like  `“`, `—`, and `…` are not included in string.punctuation, so we need to add them manually or using regex


### 📌 **Real-World:**
- For spam detection or document classification, punctuation doesn’t usually help (unless you’re analyzing writing style).

In [ ]:
# remove punctuation using string.punctuation
import string

text = "Hello!!! How are you??? I'm fine:) “Great”— yes…"
no_punc = text.translate(str.maketrans('', '', string.punctuation + '“”—…'))

print("🔵 Original Text: ", text, "\n")
print("🟢 Text Cleaned From Punctuation Using string.punctuation: ", no_punc)

🔵 Original:  Hello!!! How are you??? I'm fine:) “Great”— yes… 

🟢 Text Cleaned From Punctuation Using string.punctuation:  Hello How are you Im fine Great yes


In [ ]:
# remove punctuation using regex with Unicode support
import regex as re

text = "Hello!!! How are you??? I'm fine:) “Great”— yes…"
no_punc = re.sub(r'\p{P}+', '', text)

print("🔵 Original Text: ", text, "\n")
print("🟢 Text Cleaned From Punctuation Using Regex: ", no_punc)

🔵 Original:  Hello!!! How are you??? I'm fine:) “Great”— yes… 

🟢 Text Cleaned From Punctuation Using Regex:  Hello How are you Im fine Great yes


---

## 5️⃣ **Chat Word Treatment (e.g., GN → Good Night)**

### 🔍 **Why:**
- Chat language is full of abbreviations: "lol", "idk", "smh".

- They need to be normalized to standard English for models to understand.

### 📌 **Real-World:**
- In customer support or social media, replacing abbreviations like `"brb"` with `"be right back"` helps understand the message better.

### 🗣️ **Slang Words**
- The following two GitHub repositories provide comprehensive lists of slang words and their standard English equivalents.These resources are useful for expanding chat abbreviations (e.g., "GN" → "Good Night") during text preprocessing

    - [Repo1](https://github.com/ipekdk/abbreviation-list-english)
    - [Repo2](https://github.com/bodhwani/NLP-VIT-BOT/blob/master/slangs.csv)
    - [Comman Slang Words](https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt)

#### Replace Slang words with the help of tokenization

In [85]:
# Use spaCy for tokenization and keep punctuation marks
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")
text = "BRB guys, lol this is funny. ttyl!"
# Ensure 'nlp' is already loaded with spacy.load("en_core_web_sm")
doc = nlp(text)  # 'text' variable is already defined

# Load slang dictionary from CSV
chat_dict = pd.read_csv("slangs.csv")  # Must have columns 'Abbr', 'Fullform'
slang_map = dict(zip(chat_dict['Abbr'].str.lower(), chat_dict['Fullform']))

# Replace slangs with full forms, preserving punctuation
processed_tokens = []
for token in doc:
    token_text = token.text
    lower_token = token_text.lower()
    if lower_token in slang_map:
        processed_tokens.extend(slang_map[lower_token].split())
    else:
        processed_tokens.append(token_text)

preprocessed_text = " ".join(processed_tokens)

print("🔵 Original Text: ", text, "\n")
print("🟢 Clean Slang Words Using Tokenization: ", preprocessed_text)


🔵 Original Text:  BRB guys, lol this is funny. ttyl! 

🟢 Clean Slang Words Using Tokenization:  Be Right Back guys , Laughing Out Loud this is funny . Talk To You Later !


#### Replace Slang word with the help of Split

- you can see the difference `','` , `'.'` and `'!'` present in original text but not in preprocessed text using split but in the case of tokenization there is no missingness of `','` , `'.'` and `'!'`

In [ ]:
# just apply to your text this comprehenisve list of slang abbreviations
# Make sure to have a CSV file named 'slangs.csv' with columns 'Abbr

import pandas as pd
import regex as re  # Use the 'regex' module for Unicode property escapes

# Load slang dictionary from CSV
chat_dict = pd.read_csv("slangs.csv")  # Must have columns 'Abbr', 'Fullform'
slang_map = dict(zip(chat_dict['Abbr'].str.lower(), chat_dict['Fullform']))

# Sample chat-like text
text = "BRB guys, lol this is funny. ttyl!"

# Remove punctuation using regex for matching text
text_clean = re.sub(r'\p{P}+', '', text)

# Tokenize by whitespace
words = text_clean.split()

# Replace slangs
chat_fixed = []
for word in words:
    word_lower = word.lower()
    if word_lower in slang_map:
        chat_fixed.append(slang_map[word_lower])
    else:
        chat_fixed.append(word)

# Join the cleaned text
cleaned_text = " ".join(chat_fixed) # The join() method takes all items in an iterable and joins them into one string. A string must be specified as the separator

# Output
print("🔵 Original Text: ", text, "\n")
print("🟢 Clean From Slang Words Using Regex: ", cleaned_text)

🔵 Original:  BRB guys, lol this is funny. ttyl! 

🟢 Clean From Slang Words Using Regex:  Be Right Back guys Laughing Out Loud this is funny Talk To You Later


---

## 6️⃣ **Spelling Correction**

### 🔍 **Why:**
- Typos are common in user input (especially social media).

- Words like “beleive” won’t match “believe” in dictionaries or embeddings.

### 📌 **Real-World:**
- Spell correction boosts chatbot understanding and auto-correction in search bars.

In [ ]:
from autocorrect import Speller
from textblob import TextBlob
from spellchecker import SpellChecker


spell = Speller(lang='en')
text = "I realy love natral language prosesing."

# Autocorrect
corrected_autocorrect = spell(text)

# TextBlob
corrected_textblob = str(TextBlob(text).correct())

# SpellChecker
spellchecker = SpellChecker()
# Tokenize the text for word-level correction
tokens = text.split()
corrected_pyspellchecker = ' '.join([spellchecker.correction(word) or word for word in tokens])


print("🔵 Original Text:", text, "\n")
print("🟢 Preprocessed (autocorrect): ", corrected_autocorrect, "\n")
print("🔴 Preprocessed (TextBlob): ", corrected_textblob, "\n")
print("🟡 Preprocessed (pyspellchecker): ", corrected_pyspellchecker)

🔵 Original: I realy love natral language prosesing. 

🟢 Preprocessed (autocorrect):  I really love natural language crossing. 

🔴 Preprocessed (TextBlob):  I really love natural language pressing. 

🟡 Preprocessed (pyspellchecker):  I really love natural language prosesing.


---

## 7️⃣ **Remove Stop Words**

### 🔍 **Why:**
- Stopwords like "the", "is", "at" are function words, not content words.

- Removing them improves focus on important tokens.

### 📌 **Real-World:**
- In a movie review, “The movie was absolutely fantastic”, word “fantastic” carries the sentiment, not “the”.

In [ ]:
# Import library
import spacy

# Load the language model
nlp = spacy.load("en_core_web_sm")

# Process the text
text = "This is a sample sentence with some stop words"
doc = nlp(text)

# Remove stop words
# This line creates a list of tokens that are not stop words and joins them into a single string
# is_stop is a boolean attribute of the token object that indicates whether the token is a stop word
filtered_tokens = " ".join([token.text for token in doc if not token.is_stop]) 

# Print the text excluding stop words
print("🔵 Original Text: ", text, "\n")
print("🟢 Text After Removing Stop Words: ", filtered_tokens, "\n")

🔵 Original:  This is a sample sentence with some stop words 

🟢 Text After Removing Stop Words:  sample sentence stop words 



---

## 8️⃣ **Handling Emojis**

### 🔍 **Why:**
- Emojis are sentiment-rich tokens (😊, 😢).

- You can either remove them, convert to words, or use them as features.

### 📌 **Real-World:**
- In social media sentiment analysis, 😠 and ❤️ change the tone completely and must be considered.

In [ ]:
import emoji
import regex as re

text = "Good job!👍I’m so happy😊"
emoji_text = emoji.demojize(text)

# Remove ':' and '_' characters
emoji_text_cleaned = emoji_text.replace(':', ' ').replace('_', ' ')

print(f"🔵 Original Text: {text}\n")
print(f"🟢 Text With Handled Emojis: {emoji_text_cleaned}")


🔵 Original: Good job!👍I’m so happy😊

🟢 Text With Handled Emojis: Good job! thumbs up I’m so happy smiling face with smiling eyes 


---

## 9️⃣ **Tokenization**

### ⚠️ **Very Important Step**

### 🔍 **Why:**
Machine learning models in NLP typically require numerical input. Tokenization converts text into a format that can be converted into numerical representations (like word embeddings) that models can understand.

#### **Example:**
Let's say we have the sentence: `The quick brown fox jumps over the lazy dog.`
Word Tokenization:
> ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog", "."]

### 📌 **Real-World:**
- In the search engine, "best Italian restaurants near me" is tokenized into the words: ["best", "Italian", "restaurants", "near", "me"].

#### Tokenization with `spacy.blank("en")`

In [ ]:
import spacy

# Create a blank English NLP object (only tokenizer) No lemmatization, NER, POS & Tagging
nlp_blank = spacy.blank("en")

# Input text
text = "Hello, world! This is a test sentence. Let's see how it works."

# Tokenize using blank pipeline
doc_blank = nlp_blank(text)

# Print each token
token_list = [token.text for token in doc_blank]

print("🔵 Original Text: ", text, "\n")
print(f"🟢 After Tokenization Using spacy.blank('en'): ", token_list)

🔵 Original:  Hello, world! This is a test sentence. Let's see how it works. 

🟢 After Tokenization Using spacy.blank('en'):  ['Hello', ',', 'world', '!', 'This', 'is', 'a', 'test', 'sentence', '.', 'Let', "'s", 'see', 'how', 'it', 'works', '.']


#### Tokenization with `spacy.load("en_core_web_sm")`

In [ ]:
import spacy

# Load pretrained small English model (includes tokenizer, POS, NER, etc.)
nlp_full = spacy.load("en_core_web_sm")

# Input text
text = "Hello, world! This is a test sentence. Let's see how it works."

# Process text
doc_full = nlp_full(text)

# Print each token
token_list = [token.text for token in doc_full]

print("🔵 Original Text: ", text, "\n")
print("🟢 After Tokenization Using English Model: ", token_list)

🔵 Original:  Hello, world! This is a test sentence. Let's see how it works. 

🟢 After Tokenization Using English Model:  ['Hello', ',', 'world', '!', 'This', 'is', 'a', 'test', 'sentence', '.', 'Let', "'s", 'see', 'how', 'it', 'works', '.']


---

## 🔟 **Stemming**

### 🔍 **Why:**
- Stemming reduces words to a base/root form. It’s fast and works well in information retrieval systems.

- May produce non-words (e.g., “studies” → “studi”).

### 📌 **Real-World:**
- Used in search engines (e.g., “searching”, “searched”, “searches” → “search”) to show relevant documents.

In [ ]:
import spacy
from nltk.stem import PorterStemmer

# Load spaCy (for tokenization)
nlp = spacy.load("en_core_web_sm")

# Three well-known stemming algorithms are the Porter stemmer, Snowball stemmer, and Lancaster stemmer
# Initialize stemmer
stemmer = PorterStemmer()

# Input text
text = "Organizing events helps communities grow and stay connected."

# Tokenize using spaCy
doc = nlp(text)

# Apply stemming using NLTK
stemmed_tokens = [stemmer.stem(token.text) for token in doc]

# Output
print("🔵 Original Text: ", [token.text for token in doc], "\n")
print("🟢 After Stemming Using PorterStemmer: ", stemmed_tokens)

🔵 Original:  ['Organizing', 'events', 'helps', 'communities', 'grow', 'and', 'stay', 'connected', '.'] 

🟢 After Stemming Using PorterStemmer:  ['organ', 'event', 'help', 'commun', 'grow', 'and', 'stay', 'connect', '.']


#### **The 3 Main NLTK Stemmers + Lemmatization**

| Technique            | Aggressiveness   | Language Support                 | Speed                     | Output Quality               | Notes                                                               |
| -------------------- | ---------------- | -------------------------------- | ------------------------- | ---------------------------- | ------------------------------------------------------------------- |
| **PorterStemmer**    | ✅ Moderate       | English only                     | ✅ Fast                    | ✅ Good, widely used          | Most commonly used, balanced stemming algorithm                     |
| **LancasterStemmer** | 🔥 Very high     | English only                     | ✅ Fast                    | ❌ Over-stems words           | Very aggressive → `maximum` → `maxim`                       |
| **SnowballStemmer**  | 🟢 Balanced      | ✅ Multiple langs                 | ✅ Fast                    | ✅ Clean, improved            | Also called “Porter2” — better, more consistent than Porter         |
| **Lemmatization**    | ❌ Not aggressive | ✅ Multiple langs (WordNet/spaCy) | ⚠️ Slower (context-aware) | ✅ Real words & context-aware | Uses grammar + vocabulary, returns meaningful dictionary base forms |


In [ ]:
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Sample text
text = "The organization organized organized events efficiently."

# Tokenize
tokens = nlp(text)

# Initialize stemmers
porter = PorterStemmer() # Balanced stemmer
lancaster = LancasterStemmer() # aggressive stemmer
snowball = SnowballStemmer("english") # Also Called Porter2

# Apply each stemmer
porter_result = [porter.stem(word.text) for word in tokens]
lancaster_result = [lancaster.stem(word.text) for word in tokens]
snowball_result = [snowball.stem(word.text) for word in tokens]
lemmized_result = [word.lemma_ for word in tokens]  # Lemmatization using spaCy

print("🔵 Original Text: ", tokens, "\n")
print("🟢 PorterStemmer: ", porter_result, "\n")
print("🔴 LancasterStemmer: ", lancaster_result, "\n")
print("🟡 SnowballStemmer: ", snowball_result, "\n")
print("🟣 Lemmatization: ", lemmized_result)


🔵 Original:  The organization organized organized events efficiently. 

🟢 PorterStemmer:  ['the', 'organ', 'organ', 'organ', 'event', 'effici', '.'] 

🔴 LancasterStemmer:  ['the', 'org', 'org', 'org', 'ev', 'efficy', '.'] 

🟡 SnowballStemmer:  ['the', 'organ', 'organ', 'organ', 'event', 'effici', '.'] 

🟣 Lemmatization:  ['the', 'organization', 'organize', 'organize', 'event', 'efficiently', '.']


---

## 🔢 **Lemmatization**

### 🔍 **Why:**
- Like stemming, but linguistically accurate.

- Uses vocabulary and grammar rules to return the correct base word.

### 📌 **Real-World:**
- For grammar-sensitive tasks like question answering or summarization, “was” should be lemmatized to “be”, not “wa”.

#### **Comparison: Lemmatization in spaCy vs NLTK**
| Feature           | **spaCy**                         | **NLTK (WordNetLemmatizer)**         |
| ----------------- | --------------------------------- | ------------------------------------ |
| Accuracy          | ✅ High (uses POS + context)       | ⚠️ Medium (needs manual POS mapping) |
| Language model    | `en_core_web_sm` or larger models | WordNet dictionary                   |
| POS Tag Awareness | ✅ Automatic                       | ⚠️ Must provide POS manually         |
| Output Quality    | ✅ Real, accurate base forms       | ✅ Good, but context-blind sometimes  |
| Ease of Use       | ✅ Very easy                       | ⚠️ Slightly complex (POS conversion) |
| Performance       | ✅ Fast and optimized              | ✅ Fast but rule-based                |

In [ ]:
# Import spaCy
import spacy

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Sample text
text = "The children were running and better organized events every week."

# Process the text
doc = nlp(text)

# Lemmatize with POS awareness
lemmatized_tokens = [token.lemma_ for token in doc]

# Output
print("🔵 Original Text: ", tokens, "\n")
print("🟢 After Lemmatization: ", " ".join(lemmatized_tokens), "\n")


🔵 Original:  The organization organized organized events efficiently. 

🟢 After Lemmatization:  the child be run and well organize event every week . 



In [ ]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

lemmatizer = WordNetLemmatizer()

# POS tag conversion
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

text = "The children were running and better organized events every week."
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
result = [tokens.text for tokens in doc]
tagged = pos_tag(result)

lemmas = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged]
print("NLTK Lemmas:", lemmas)

NLTK Lemmas: ['The', 'child', 'be', 'run', 'and', 'good', 'organize', 'event', 'every', 'week', '.']


---

## ✅ Summary Table

| Step                | Purpose                                                                 | Use Case Examples                                     |
|---------------------|-------------------------------------------------------------------------|--------------------------------------------------------|
| Lowercasing          | Normalize text casing                                                   | Text classification, search                           |
| Remove HTML Tags     | remove unwanted HTML tags                                                | Web scraping, email cleaning                          |
| Remove URLs          | Remove unnecessary web links                                             | Social media, forums                                  |
| Remove Punctuation   | Clean punctuation noise                                                  | Preprocessing for BoW/TF-IDF                          |
| Chat Word Treatment  | Convert slang to standard English like GD to Good Night                  | Chatbots, social media analysis                       |
| Spelling Correction  | Fix typos for vocabulary consistency                                     | User reviews, text input                              |
| Remove Stop Words    | Focus on meaningful words                                                 | Summarization, topic modeling                         |
| Handle Emojis        | Preserve or convert emojis based on task                                | Sentiment analysis                                    |
| Tokenization         | Break down text into tokens                                              | All NLP tasks                                         |
| Stemming             | Reduce word forms to base/root                                           | Search engines, topic modeling                        |
| Lemmatization        | Get accurate root word using grammar                                     | Parsing, QA systems, deep learning models             |
